In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('https://practicum-content.s3.us-west-1.amazonaws.com/datasets/travel_insurance_us.csv')

train, valid = train_test_split(data, test_size=0.25, random_state=12345)

features_train = train.drop('Claim', axis=1)
target_train = train['Claim']
features_valid = valid.drop('Claim', axis=1)
target_valid = valid['Claim']

print(features_train.shape)
print(features_valid.shape)


(37995, 10)
(12665, 10)


La técnica OHE nos permite transformar características categóricas en características numéricas en dos pasos:

    Agrega una columna separada para cada valor de función.
    Si la categoría se ajusta a la observación, se asigna 1, de lo contrario, se asigna 0.

Las nuevas columnas (Gender_F, Gender_M, Gender_None) se denominan variables dummy.

In [7]:
import pandas as pd

print(pd.get_dummies(data['Gender']).head(15)) 

'''
drop_first=False, 
dummy_na=False
'''

        M    NaN
0    True  False
1   False   True
2   False   True
3   False   True
4    True  False
5   False   True
6   False  False
7   False   True
8   False   True
9   False   True
10   True  False
11  False   True
12  False   True
13  False   True
14   True  False


(Si usamos dummy_na=True) Hemos agregado tres columnas nuevas a nuestra tabla, pero su alta correlación confundirá a nuestro modelo. Para evitar esto, podemos eliminar con seguridad cualquier columna, ya que sus valores se pueden deducir fácilmente de las otras dos columnas (tiene 1 donde las otras dos columnas tienen ceros y tiene ceros en el resto). De esta manera, no caeremos en la trampa dummy.

Para eliminar la columna, llama a la función pd.get_dummies() junto con el parámetro drop_first. Si pasas drop_first=True entonces se elimina la primera columna. De lo contrario, es drop_first=False por defecto y no se descartan columnas.

BUSCAR QUÉ ES LA TRAMPA DUMMY, CUANDO SE CAE EN ELLA Y CÓMO EVITARLA

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

#data = pd.read_csv('/datasets/travel_insurance_us.csv')

data_ohe = pd.get_dummies(data, drop_first=True)
target = data_ohe['Claim']
features = data_ohe.drop('Claim', axis=1)

features_train, features_valid, target_train, target_valid = train_test_split(features, target, train_size=0.75, random_state=12345)

model = LogisticRegression(solver='liblinear', random_state=12345)
model.fit(features_train, target_train)

print('¡Entrenado!')

¡Entrenado!


In [13]:
model.predict(features_valid)
score = model.score(features_valid, target_valid) 
print(score)

0.9866561389656534


| Modelo de ML | ¿Necesita Codificación? | Tipo de Codificación más Apropiada | Razón |
| :--- | :--- | :--- | :--- |
| **Regresión Lineal/Logística** | **Sí** | **One-Hot Encoding (OHE)** | Asume una relación lineal. OHE evita introducir una relación de orden artificial (ej. 1 < 2 < 3) entre categorías nominales. |
| **Máquinas de Soporte Vectorial (SVM)** | **Sí** | **One-Hot Encoding (OHE)** | Se basa en el cálculo de distancias. OHE asegura distancias equitativas entre categorías nominales. |
| **k-Vecinos Más Cercanos (kNN)** | **Sí** | **One-Hot Encoding (OHE)** | Se basa completamente en distancias. OHE es crucial para evitar distancias engañosas entre categorías nominales. |
| **Redes Neuronales** | **Sí** | **One-Hot Encoding (OHE)** o **Embeddings** | Trabajan con gradientes y ponderaciones. OHE es el estándar; Embeddings se usan para alta cardinalidad. |
| **Árboles de Decisión** | **No** (Directamente) | **Ordinal Encoding** (Si hay orden) / **Target Encoding** | No son sensibles a la escala o distancias. La Codificación Ordinal es aceptable para categorías con orden natural. |
| **Random Forest** | **No** (Directamente) | **Ordinal Encoding** (Si hay orden) / **Target Encoding** | Al ser un conjunto de árboles, tienen la misma flexibilidad, aceptando Label Encoding/Ordinal. |
| **Gradient Boosting (XGBoost, LightGBM)** | **No** (Directamente) | **Label Encoding** (Aceptable) | Implementaciones modernas pueden manejar números enteros o texto directamente; no requieren OHE gracias a la naturaleza del árbol. |
| **K-Means (Clustering)** | **Sí** | **One-Hot Encoding (OHE)** | Se basa en la distancia para formar clústeres. OHE es fundamental para variables nominales. |

In [ ]:
Ordinal Encoder

1. Crea una instancia de esta clase.

In [15]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()

Antes de usar el codificador, debes ajustarlo a tus datos para que reconozca qué variables son categóricas. Para hacerlo, llama al método fit() y pásale los datos (de la misma manera que lo hacemos cuando entrenamos un modelo).

In [16]:
encoder.fit(data)

OrdinalEncoder()

Utiliza el método transform(). Los datos transformados se almacenarán en la variable data_ordinal .

In [17]:
data_ordinal = encoder.transform(data)
data_ordinal = pd.DataFrame(encoder.transform(data), columns=data.columns)


Si necesitas transformar los datos solo una vez, como en nuestro caso, puedes llamar al método fit_transform() en su lugar.  Este combina fit() y transform().

In [18]:
data_ordinal = pd.DataFrame(encoder.fit_transform(data), columns=data.columns)

In [19]:
display(data_ordinal)

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commission (in value),Gender,Age
0,9.0,0.0,1.0,24.0,0.0,14.0,121.0,602.0,486.0,1.0,37.0
1,7.0,1.0,1.0,10.0,0.0,52.0,61.0,470.0,0.0,NaN,34.0
2,7.0,1.0,1.0,10.0,0.0,253.0,16.0,735.0,0.0,NaN,34.0
3,7.0,1.0,1.0,10.0,0.0,8.0,54.0,121.0,0.0,NaN,34.0
4,9.0,0.0,1.0,24.0,0.0,7.0,24.0,602.0,486.0,1.0,32.0
...,...,...,...,...,...,...,...,...,...,...,...
50655,7.0,1.0,1.0,1.0,0.0,57.0,85.0,470.0,0.0,NaN,34.0
50656,7.0,1.0,1.0,10.0,0.0,9.0,118.0,343.0,0.0,NaN,34.0
50657,7.0,1.0,1.0,1.0,0.0,147.0,3.0,792.0,0.0,NaN,34.0
50658,7.0,1.0,1.0,1.0,0.0,86.0,124.0,457.0,0.0,NaN,22.0


In [22]:
from sklearn.tree import DecisionTreeClassifier

target = data_ordinal['Claim']
features = data_ordinal.drop('Claim', axis=1)
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345
)

model = DecisionTreeClassifier(random_state=12345)
model.fit(features_train, target_train)


score = model.score(features_valid, target_valid) 
print(score)

0.9866561389656534


In [ ]:
Codificación ordinal (cuando hay valores en las categorías)

In [ ]:
temperature_dict = {'cold': 0, 'warm': 1, 'hot': 2}
df['temperature'] = df['temperature'].map(temperature_dict)